In [1]:
# 해당 소스는 예시이며, one_hot_train과 one_hot_test의 부분은 본인들의 data set으로 활용해야함

'''
예를들어, 7일전 24시간 온도로 오늘 24시간 온도를 예측하는 모델이라면,
i_call_x : 이전 기간의 온도값 (24시간어치)
i_call_y : 오늘 기간의 온도값 (24시간어치)
'''
# 이와 같이 mase에 실제 정답값으로 값을 전달하여 사용하게됨.

y_columns = ['i_call_x','i_call_y']
x_columns = list(set(one_hot_train.columns) - set(['i_call_y']))

one_hot_train_x = one_hot_train[x_columns]
one_hot_train_y = one_hot_train[y_columns]
one_hot_test_x = one_hot_test[x_columns]
one_hot_test_y = one_hot_test[y_columns]

NameError: name 'one_hot_train' is not defined

In [3]:
from tensorflow import keras
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout, LeakyReLU, ReLU, TimeDistributed, Bidirectional, RepeatVector
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf

In [5]:
model = Sequential()
model.add(LSTM(1024, return_sequences=True, input_shape=(24,10)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(512, return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 24, 1024)          4239360   
_________________________________________________________________
lstm_1 (LSTM)                (None, 24, 512)           3147776   
_________________________________________________________________
time_distributed (TimeDistri (None, 24, 1)             513       
Total params: 7,387,649
Trainable params: 7,387,649
Non-trainable params: 0
_________________________________________________________________


In [7]:
from custom_loss_2_true import custom_loss

# 실제 모델 활용 예시
loss = custom_loss()
# 상위 모듈에서 이미 작성된 소스로 y_true와 y_pred는 자동으로 모델에서 넘어가기때문에, 넘겨주는 값으로만 잘 설정해주면 소스 동작에는 이상이 없음.
model.compile(loss=loss.mase, optimizer=Adam(lr=0.001))
model.fit(one_hot_train_x.values, one_hot_train_y.values, batch_size=200, epochs=3000, verbose=2, shuffle=True)

NameError: name 'one_hot_train_x' is not defined

In [ ]:
'''
만약 필자의 mase 소스를 수정하여, 더 많은 값을 활용하고 싶다면
@tf.function
train_step을 사용하여 직접 loss에 들어가는 값들을 관장해주어야함.
'''

# 아래의 소스를 활용하여 수정할 수 있겠음. 필자는 현재 이렇게 사용하고 있지는 않아서, 사용하게 될 시에 추가하여 올리도록 하겠음.
@tf.function
def train_step(inputs,labels,loss_fn,optimizer):
    loss = 0
    encoder_input, decoder_input = inputs
    with tf.GradientTape() as tape:
        enc_output, enc_hidden, enc_c = encoder(encoder_input)

        for t in range(0, decoder_input.shape[1]):
            dec_input = tf.reshape(decoder_input[:,t],(-1,1,decoder_input.shape[2]))
            predictions, attention_weights = decoder(dec_input, enc_output, enc_hidden, enc_c)
            loss += loss_fn(tf.reshape(labels[:,t],(-1,1,1)), predictions)
    batch_loss = tf.reduce_mean(loss)
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss,variables)
    optimizer.apply_gradients(zip(gradients,variables))

    return batch_loss

loss_fn = loss
optimizer = tf.keras.optimizers.Adam(0.001)